In [1]:
from ase import io
from ase.visualize import view
from ipywidgets import HBox
from ase import Atoms
from ase.build import separate

In [2]:
#Insert molecule file to use here
molecule = io.read('Organofluorine_files/PFDMA_straight.xyz')

#Which metal is being added
metal = 'Ag'

#Distance from end carbon for first metal atom  (silver has an atomic radius of ~1.75 Å)
min_dist = 1.30

#Spacings for silver atoms (Angstroms)
spacing = 0.2

#Number of distances
no_dists = 20

#Core name for files
fname = 'PFOMA_Ag_2deg'

#Trajectory of approach (number of degrees rotated from central bond axis)
rot = 2

In [3]:
# @hidden_cell

# Find the carbon with 3 fluorines attached

#Set max C-F bond length
CF_max=1.4

#Set max C-C bond length
CC_max=1.6

#Give end_carbon a value not in atom list to confirm when atom is found
end_carbon = -1

#Obtain chemical symbols for atom list
sym = molecule.get_chemical_symbols()

#Check neighbours for each atom to find CF3
for i in range(len(molecule)):
    
    #Number of fluorines
    nF = 0
    
    if sym[i]=='C':
        for j in range(len(molecule)):
            if sym[j]=='F' and molecule.get_distance(i,j) < CF_max:
                nF = nF + 1
    if nF==3:
        assert(end_carbon==-1) # Fail if CF3 has already been found
        end_carbon = i
        
# check if a CF3 carbon has been found
assert(end_carbon>=0) 
print(end_carbon)

36


In [4]:
# Find atom to create vector with the end carbon

vector_atom = -1
CC_min = 1.0 # avoid locating same atom twice

for i in range(len(molecule)):
    
    if sym[i]=='C' and molecule.get_distance(i,end_carbon) < CC_max and molecule.get_distance(i,end_carbon) > CC_min:
       # assert(vector_atom==-1) #Fail if atom has already been found
        vector_atom = i
        
assert(vector_atom>=0)
    
print(vector_atom)

33


In [5]:
#Calculate vector between end carbon and vector atom
vector = molecule.get_positions()[end_carbon] - molecule.get_positions()[vector_atom]

#Rotate the vector to be along the x-axis
molecule.rotate(vector,(1,0,0))

end_carbon_pos = molecule.get_positions()[end_carbon]

view(molecule, viewer = 'ngl' )

In [6]:
#change approach trajectory
molecule.rotate(rot,'z')

end_carbon_pos = molecule.get_positions()[end_carbon]

view(molecule, viewer = 'ngl' )

'''
filename = fname + '.xyz'
 
#save xyz file 
io.write(filename, molecule, format = 'xyz')
'''

In [ ]:
# Add silver atom and save xyz file for each distance

for i in range(no_dists):
    
    #set distance between end carbon and silver atom
    dist = min_dist + (0.2 * i)
    #print (dist)
    #Append a silver atom to the atoms list
    molecule.append(metal)

    #Place the silver atom at the minimum distance from the end carbon
    molecule.positions[-1] = end_carbon_pos + (dist,0,0)
    
    filename = fname + '_' + str(round((min_dist + i * 0.20), 2)) + '.xyz'
 
    #save xyz file 
    io.write(filename, molecule, format = 'xyz')
    
    #Remove silver atom for next iteration
    del molecule[-1]


In [ ]:
for i in range(no_dists):
    
    #set distance between end carbon and silver atom
    dist = min_dist + (0.2 * i)
    #print (dist)
    #Append a silver atom to the atoms list
    molecule.append(metal)

    #Place the silver atom at the minimum distance from the end carbon
    molecule.positions[-1] = end_carbon_pos + (dist,0,0)
    
view(molecule, viewer = 'ngl' )